載入套件

In [405]:
%matplotlib inline
#確保一致性
%env KERAS_BACKEND = tensorflow 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import mnist    #資料
from keras.utils import np_utils    #轉為分類問題
from keras.utils import to_categorical 
from keras.models import Sequential #神經網路相關套件  標準
from keras.layers import Dense, Activation
from keras.optimizers import SGD, Adam, RMSprop    #隨機調整
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate, add
from ipywidgets import interact_manual 
from sklearn.model_selection import train_test_split#分割資料集

env: KERAS_BACKEND=tensorflow


In [406]:
#df[''] = [e for e in df['']]

#### 原始資料

In [410]:
mrt1 = pd.read_csv('C:/Users/user/Desktop/python/final/mrt1.csv',engine = "python")
mrt2 = pd.read_csv('C:/Users/user/Desktop/python/final/mrt2.csv',engine = "python")
mrt3 = pd.read_csv('C:/Users/user/Desktop/python/final/mrt3.csv',engine = "python")
mrt = pd.concat([mrt1,mrt2,mrt3],axis=0)  #串接dataframe

In [411]:
mrt.head()

,卡號,卡種,進站時間,出站時間,進站站名,出站站名
0,B1EF20523AF10C3389056806C93DD5BE,1,2018/3/15 23:23,2018/3/16 00:00,行天宮,海山
1,B0BCA6332F559DA7F3BAD948933386B9,1,2018/3/15 23:08,2018/3/16 00:00,西湖,頭前庄
2,2E5013A0819F671A73979766C9583BAD,1,2018/3/15 23:30,2018/3/16 00:00,士林,西門
3,7215C2AC14E7AFB9885D30353824D5F1,1,2018/3/15 23:41,2018/3/16 00:00,西門,新埔
4,8004EB8D24B387203C0CD0171B4868F5,6,2018/3/15 23:23,2018/3/16 00:00,士林,新埔


#### 合理進出篩選
dfq 已篩選掉 怪異地點 : 第一次出站站點周圍1公里無學校

(輔大、唭哩岸、大湖公園、竹圍、頂埔)

In [409]:
dfq = pd.read_csv('C:/Users/user/Desktop/python/final/df_1_del.csv',engine = "python",encoding ='utf-8')
dfq = dfq.drop(['Unnamed: 0'],axis = 1) 
card_id = pd.unique(dfq['卡號'])  #取出卡號
dfq.head()

,卡號,卡種,進站時間,出站時間,進站站名,出站站名
0,00001828EC89F17789056806C93DD5BE,1,2018/3/15 08:35,2018/3/15 08:47,景安,古亭
1,00001828EC89F17789056806C93DD5BE,1,2018/3/15 18:21,2018/3/15 18:40,古亭,景安
2,00003BB606A8D4471CE3A1303DC0F97F,1,2018/3/15 06:20,2018/3/15 06:31,唭哩岸,北投
3,00003BB606A8D4471CE3A1303DC0F97F,1,2018/3/15 22:07,2018/3/15 22:14,北投,唭哩岸
4,0000A959F2E22684577055C745EF9729,1,2018/3/15 18:21,2018/3/15 18:38,台北車站,信義安和


由於檔案過大，將資料切分後再進行運算

篩選合理進出 :

第一次進站 = 最後一次出站       

第一次出站 = 第二次進站

In [279]:
l = []
for j in range(0,1000):
    card_id2 = card_id[range((j*446),(j+1)*446)]
    dfq2 = dfq.loc[ dfq['卡號'].isin(card_id2)]   #將 dataframe 切成1000份執行
    for i in card_id2:
        a = dfq2[dfq2.卡號 == i]  #從unique叫出卡號
        l.append((a.iloc[0,4]==a.iloc[-1,5]) & (a.iloc[0,5]==a.iloc[1,4]) )
                 #頭進站 == 尾出站          #第一次出站 == 第二次進站446601

將餘數資料補回

In [282]:
card_id3 = card_id[446000:446601]
dfq3 = dfq.loc[ dfq['卡號'].isin(card_id3)]
for i in card_id3:
    a = dfq3[dfq3.卡號 == i]  #從unique叫出卡號
    l.append((a.iloc[0,4]==a.iloc[-1,5]) & (a.iloc[0,5]==a.iloc[1,4]) )

取出合理進出資料、輸出

In [295]:
dfq4 = dfq.loc[dfq['卡號'].isin(card_id[l])]
dfq5 = dfq.loc[dfq['卡號'].isin(card_id[np.logical_not(l)])] 
dfq4.to_csv('C:/Users/user/Desktop/合理進出.csv')
dfq5.to_csv('C:/Users/user/Desktop/不合理3.csv')

#### 合理旅次篩選

合理旅次時間 : 上下午皆有旅次

In [416]:
dfr = card1 = pd.read_csv('C:/Users/user/Desktop/python/final/合理進出.csv',engine = "python",encoding ='utf-8')
#dfr = dfr.drop(['Unnamed: 0','Unnamed: 0.1'],axis = 1) 
dfr = dfr.drop(['Unnamed: 0'],axis = 1) 
dfcard = pd.unique(dfr.卡號)

In [417]:
dfr.head()

,卡號,卡種,進站時間,出站時間,進站站名,出站站名
0,00001828EC89F17789056806C93DD5BE,1,2018/3/15 08:35,2018/3/15 08:47,景安,古亭
1,00001828EC89F17789056806C93DD5BE,1,2018/3/15 18:21,2018/3/15 18:40,古亭,景安
2,00003BB606A8D4471CE3A1303DC0F97F,1,2018/3/15 06:20,2018/3/15 06:31,唭哩岸,北投
3,00003BB606A8D4471CE3A1303DC0F97F,1,2018/3/15 22:07,2018/3/15 22:14,北投,唭哩岸
4,0000A959F2E22684577055C745EF9729,1,2018/3/15 18:21,2018/3/15 18:38,台北車站,信義安和


將時間格式

由  2018/3/15 08:35   轉為 08:35

In [47]:
dfr['進站時間'] = [ i[10:15] for i in dfr['進站時間'] ]
dfr['出站時間'] = [ i[10:15] for i in dfr['出站時間'] ]   #留下時間

篩選

第一次進站時間早於早上9點

最後一次進站時間晚於下午3點半

同樣將資料切分 分1000次跑

In [55]:
l = []
for j in range(0,1000):
    dfcard2 = dfcard[range((j*284),(j+1)*284)]
    dfr2 = dfr.loc[ dfr['卡號'].isin(dfcard2)]
    for i in dfcard2:
        a = dfr2[dfr2.卡號 == i]  #從unique叫出卡號
        b = a.sort_values("進站時間")
        l.append((b.iloc[0,4] < '09:00') & (b.iloc[-1,4] > '15:30'))

In [58]:
dfcard3 = dfcard[284000:284077]
dfr3 = dfr.loc[ dfr['卡號'].isin(dfcard3)]
for i in  dfcard3:
    a = dfr3[dfr3.卡號 == i]  #從unique叫出卡號
    b = a.sort_values("進站時間")
    l.append((b.iloc[0,4] < '09:00') & (b.iloc[-1,4] > '15:30'))

In [63]:
dfr4 = dfr.loc[dfr['卡號'].isin(dfcard[l])]
dfr5 = dfr.loc[dfr['卡號'].isin(dfcard[np.logical_not(l)])] 
#df4 = df3.sort_values(['卡號','進站時間'])
#df7 = df2.sort_values(['卡號','進站時間'])
dfr4.to_csv('C:/Users/user/Desktop/合理旅次.csv')
dfr5.to_csv('C:/Users/user/Desktop/不合理4.csv')

#### 整理成訓練格式

In [91]:
dftr = pd.read_csv('C:/Users/user/Desktop/python/final/合理旅次.csv',engine = "python",encoding = 'utf-8')
dftr = dftr.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'],axis = 1) 
cardu = pd.unique(df7.卡號)  

In [467]:
dftr.head()

,卡號,卡種,進站時間,出站時間,進站站名,出站站名
0,00001828EC89F17789056806C93DD5BE,1,08:35,08:47,景安,古亭
1,00001828EC89F17789056806C93DD5BE,1,18:21,18:40,古亭,景安
2,00003BB606A8D4471CE3A1303DC0F97F,1,06:20,06:31,唭哩岸,北投
3,00003BB606A8D4471CE3A1303DC0F97F,1,22:07,22:14,北投,唭哩岸
4,000151D89ED0D51573979766C9583BAD,1,08:48,09:02,古亭,大坪林


In [110]:
ca = []
for j in range(0,100):
    cardu2 = cardu[range((j*2288),(j+1)*2288)]
    dftr2 = dftr.loc[ dftr['卡號'].isin(cardu2)]
    for i in cardu2:
        a = dftr2[dftr2.卡號 == i]  #從unique叫出卡號
        ca.append([a.iloc[0,4],a.iloc[0,5],a.iloc[0,2],a.iloc[0,3],a.iloc[-1,2],a.iloc[-1,3],a.iloc[0,1]])

In [100]:
cardu3 = cardu[228800:228890]
dftr3 = dftr.loc[ dftr['卡號'].isin(cardu3)]
for i in cardu3:
    a = dftr3[dftr3.卡號 == i]  #從unique叫出卡號
    ca.append([a.iloc[0,4],a.iloc[0,5],a.iloc[0,2],a.iloc[0,3],a.iloc[-1,2],a.iloc[-1,3],a.iloc[0,1]])

In [102]:
df8 = pd.DataFrame(ca,columns= ['第一次進站地點', '第一次出站地點','第一次進站時間',
                               '第一次出站時間','最後一次進站時間','最後一次出站時間','卡種'])
df9 = df8.sort_values(['第一次進站時間','卡種'])

In [109]:
df9.to_csv('C:/Users/user/Desktop/合理旅次_訓練格式.csv')

#### 處理時間、站點格式

In [115]:
df9 = pd.read_csv('C:/Users/user/Desktop/python/final/合理旅次_訓練格式.csv',engine = "python",encoding = 'utf-8')
df9 = df9.drop(['Unnamed: 0'],axis = 1) 
stacode = pd.read_csv('C:/Users/user/Desktop/python/final/station_code2.csv',engine = "python")

In [471]:
stacode.head()

,代號,站名
0,1,動物園
1,2,木柵
2,3,萬芳社區
3,4,萬芳醫院
4,5,辛亥


將捷運站名轉為代號

使用字典格式處理資料

In [119]:
stacode2 = stacode.set_index('站名').T.to_dict('int')
stacode3 = stacode2["代號"]
df9['第一次進站地點'] = [ stacode3[i] for i in df9['第一次進站地點']]
df9['第一次出站地點'] = [ stacode3[i] for i in df9['第一次出站地點']]

將時間轉數字 ( 0 - 1)

In [121]:
sta =  df9.iloc[0:len(df9),1:2] 
sta.head()
t_1 = [(int(e[: 2])*60 + int(e[3: 5]))/1440 for e in df9['第一次進站時間']]
t_2 = [(int(e[: 2])*60 + int(e[3: 5]))/1440 for e in df9['第一次出站時間']]
t_3 = [(int(e[: 2])*60 + int(e[3: 5]))/1440 for e in df9['最後一次進站時間']]
t_4 = [(int(e[: 2])*60 + int(e[3: 5]))/1440 for e in df9['最後一次出站時間']]
df9['第一次進站時間']=t_1
df9['第一次出站時間']=t_2
df9['最後一次進站時間']=t_3
df9['最後一次出站時間']=t_4

In [470]:
df9.head()

,第一次進站地點,第一次出站地點,第一次進站時間,第一次出站時間,最後一次進站時間,最後一次出站時間,卡種
0,83,24,0.229861,0.283333,0.709722,0.744444,1
1,61,29,0.230556,0.252083,0.656944,0.663194,1
2,91,11,0.230556,0.273611,0.837500,0.863889,1
3,91,107,0.231250,0.279167,0.711111,0.743750,1
4,83,66,0.231250,0.277083,0.694444,0.722222,2


In [514]:
#stacode['num'] = [(stacode['代號']/108)[i] for i in range(0,108)]  #標準化 目前沒用

In [123]:
df9.to_csv('C:/Users/user/Desktop/合理旅次_訓練格式_時間地點.csv')

#### 非學生通勤資料 type1234 格式處理

重複上述步驟即可

In [158]:
df22 = pd.read_csv('C:/Users/user/Desktop/python/final/4個藍色禁止標誌.csv',engine = "python",encoding = 'utf-8')
df22 = df22.drop(['Unnamed: 0'],axis = 1) 
df23 = df22.sort_values(['進站時間'])
cardnum = pd.unique(df23.卡號)

In [140]:
# dfcardg = np.random.choice(dfcardt,23679)

In [194]:
cat = []
for j in range(0,1000):
    cardnum2 = cardnum[range((j*324),(j+1)*324)]
    df24 = df23.loc[ df23['卡號'].isin(cardnum2)]
    print(j)
    for i in cardnum2:
        a = df24[df24.卡號 == i]  #從unique叫出卡號
        cat.append([a.iloc[0,4],a.iloc[0,5],a.iloc[0,2],a.iloc[0,3],a.iloc[-1,2],
                    a.iloc[-1,3],a.iloc[0,1],a.iloc[0,6]])

In [179]:
cardnum3 = cardnum[324000:324955]
df25 = df23.loc[ df23['卡號'].isin(cardnum3)]
for i in cardnum3:
    a = df25[df25.卡號 == i]  #從unique叫出卡號
    cat.append([a.iloc[0,4],a.iloc[0,5],a.iloc[0,2],a.iloc[0,3],a.iloc[-1,2],
                a.iloc[-1,3],a.iloc[0,1],a.iloc[0,6]])

In [188]:
df26 = pd.DataFrame(cat,columns= ['第一次進站地點', '第一次出站地點','第一次進站時間',
                               '第一次出站時間','最後一次進站時間','最後一次出站時間','卡種','type'])
df27 = df26.sort_values(['第一次進站時間','type'])

In [190]:
stacode = pd.read_csv('C:/Users/user/Desktop/python/final/station_code2.csv',engine = "python")
stacode2 = stacode.set_index('站名').T.to_dict('int')
stacode3 = stacode2["代號"]
df27['第一次進站地點'] = [ stacode3[i] for i in df27['第一次進站地點']]
df27['第一次出站地點'] = [ stacode3[i] for i in df27['第一次出站地點']]

In [191]:
t_1 = [(int(e[: 2])*60 + int(e[3: 5]))/1440 for e in df27['第一次進站時間']]
t_2 = [(int(e[: 2])*60 + int(e[3: 5]))/1440 for e in df27['第一次出站時間']]
t_3 = [(int(e[: 2])*60 + int(e[3: 5]))/1440 for e in df27['最後一次進站時間']]
t_4 = [(int(e[: 2])*60 + int(e[3: 5]))/1440 for e in df27['最後一次出站時間']]
df27['第一次進站時間']=t_1
df27['第一次出站時間']=t_2
df27['最後一次進站時間']=t_3
df27['最後一次出站時間']=t_4

In [473]:
df27.head()

,第一次進站地點,第一次出站地點,第一次進站時間,第一次出站時間,最後一次進站時間,最後一次出站時間,卡種,type
0,30,52,0.229167,0.265278,0.313194,0.329861,129,4
1,83,41,0.229861,0.272917,0.745833,0.769444,2,3
2,99,10,0.231250,0.261111,0.715278,0.725694,2,3
3,98,103,0.231944,0.263194,0.668750,0.685417,1,3
4,98,67,0.231944,0.265972,0.754861,0.766667,6,3


In [193]:
df27.to_csv('C:/Users/user/Desktop/type1234_訓練格式_時間地點.csv')

### 取出資料 
dfr : 經篩選過後之合理旅次 <br>
dfn : 篩選掉的資料 

In [448]:
dfr = pd.read_csv('C:/Users/user/Desktop/python//final/合理旅次_訓練格式_時間地點.csv',engine = "python",encoding = 'utf-8')
dfn = pd.read_csv('C:/Users/user/Desktop/python//final/type1234_訓練格式_時間地點.csv',engine = "python",encoding = 'utf-8')
dfr = dfr.drop(['Unnamed: 0'],axis = 1) 
dfn = dfn.drop(['Unnamed: 0'],axis = 1) 

In [474]:
dfr.head()

,第一次進站地點,第一次出站地點,第一次進站時間,第一次出站時間,最後一次進站時間,最後一次出站時間,卡種
0,83,24,0.229861,0.283333,0.709722,0.744444,1
1,61,29,0.230556,0.252083,0.656944,0.663194,1
2,91,11,0.230556,0.273611,0.837500,0.863889,1
3,91,107,0.231250,0.279167,0.711111,0.743750,1
4,83,66,0.231250,0.277083,0.694444,0.722222,2


In [475]:
dfn.head()

,第一次進站地點,第一次出站地點,第一次進站時間,第一次出站時間,最後一次進站時間,最後一次出站時間,卡種,type
0,30,52,0.229167,0.265278,0.313194,0.329861,129,4
1,83,41,0.229861,0.272917,0.745833,0.769444,2,3
2,99,10,0.231250,0.261111,0.715278,0.725694,2,3
3,98,103,0.231944,0.263194,0.668750,0.685417,1,3
4,98,67,0.231944,0.265972,0.754861,0.766667,6,3


#### 將合理旅次資料標籤 ####

訓練資料:測試資料為7:3

In [449]:
#學生卡
dfr1 = dfr[dfr['卡種']==6]
dfr1 = dfr1.drop(['卡種'],axis = 1)
dfr1y = np.array([1]*dfr1.shape[0])
dfr1t,dfr1test,dfr1yl,dfr1ytest= train_test_split(dfr1,dfr1y,test_size=0.3)   #切分資料為 train test

#普通卡
dfr2 = dfr[dfr['卡種']==1]
dfr2 = dfr2.drop(['卡種'],axis = 1) 
dfr2y = np.array([2]*dfr2.shape[0])
dfr2t,dfr2test,dfr2yl,dfr2ytest= train_test_split(dfr2,dfr2y,test_size=0.3)

#其他卡種
dfr3 = dfr[(dfr['卡種']!=1) & (dfr['卡種']!=6)]
dfr3 = dfr3.drop(['卡種'],axis = 1) 
dfr3y = np.array([0]*dfr3.shape[0])
dfr3t,dfr3test,dfr3yl,dfr3ytest= train_test_split(dfr3,dfr3y,test_size=0.3)

In [423]:
#### 將非學生通勤旅次資料標籤

##type 1 
#dfn1 = dfn[dfn['type'] == 1]
#dfn1 = dfn1.drop(['卡種','type'],axis = 1) 
#dfn1y = np.array([0]*dfn1.shape[0])
#dfn1t,q,dfn1yl,q = train_test_split(dfn1,dfn1y,test_size=0.9)
##type 2 
#dfn2 = dfn[dfn['type'] == 2]
#dfn2 = dfn2.drop(['卡種','type'],axis = 1) 
#dfn2y = np.array([0]*dfn2.shape[0])
#dfn2t,q,dfn2yl,q = train_test_split(dfn2,dfn2y,test_size=0.9)
##type 3 
#dfn3 = dfn[dfn['type'] == 3]
#dfn3 = dfn3.drop(['卡種','type'],axis = 1) 
#dfn3y = np.array([0]*dfn3.shape[0])
#dfn3t,q,dfn3yl,q = train_test_split(dfn3,dfn3y,test_size=0.9)
##type 4 
#dfn4 = dfn[dfn['type'] == 4]
#dfn4 = dfn4.drop(['卡種','type'],axis = 1) 
#dfn4y = np.array([0]*dfn4.shape[0])
#dfn4t,q,dfn4yl,q = train_test_split(dfn4,dfn4y,test_size=0.9)

In [450]:
x_train = pd.concat([dfr1t,dfr2t,dfr3t],axis=0)    #部分旅次合理卡 
y_train = np.concatenate([dfr1yl,dfr2yl,dfr3yl])
x_test =  pd.concat([dfr1test,dfr2test,dfr3test],axis=0)   #其他部分合理卡號
y_test = np.concatenate([dfr1ytest,dfr2ytest,dfr3ytest])
y_train = np_utils.to_categorical(y_train, 3)  
y_test = np_utils.to_categorical(y_test, 3)

### Training

輸入分為學生卡、一般卡及其他卡種，使機器判定為何卡種

輸入包括  第一次進站地點、第一次出站地點、第一次進站時間、第一次出站時間、最後一次進站時間、最後一次出站時間

輸出為 標籤 0(其他卡種) 1(學生卡) 2(一般卡)

#### Model 1 

使用簡單的類神經網絡

In [493]:
L1 = 16 #第一層
L2 = 32  #第二層
model_1 = Sequential()
model_1.add(Dense(L1, input_dim=6)) 
model_1.add(Activation('relu'))      #選擇常用的relu    #測試後relu結果較好
model_1.add(Dense(L2))                 
model_1.add(Activation('relu'))
model_1.add(Dense(3))                #output 分類的 3維
model_1.add(Activation('sigmoid'))   
model_1.compile(loss='binary_crossentropy',  
              optimizer=Adam(lr=0.05), metrics=["accuracy"])  #選擇常用optimizer Adam
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_163 (Dense)            (None, 16)                112       
_________________________________________________________________
activation_73 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_164 (Dense)            (None, 32)                544       
_________________________________________________________________
activation_74 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_165 (Dense)            (None, 3)                 99        
_________________________________________________________________
activation_75 (Activation)   (None, 3)                 0         
Total params: 755
Trainable params: 755
Non-trainable params: 0
_________________________________________________________________


In [494]:
model_t = model_1.fit(x_train, y_train, 
                    batch_size=128, epochs=3, verbose=1,
                    validation_data = (x_test, y_test))

Train on 160222 samples, validate on 68668 samples
Epoch 1/3
160222/160222 [==============================] - 6s 38us/step - loss: 0.4828 - acc: 0.8185 - val_loss: 0.4327 - val_acc: 0.8202
Epoch 2/3
160222/160222 [==============================] - 4s 26us/step - loss: 0.4334 - acc: 0.8202 - val_loss: 0.4339 - val_acc: 0.8202
Epoch 3/3
160222/160222 [==============================] - 4s 25us/step - loss: 0.4332 - acc: 0.8202 - val_loss: 0.4329 - val_acc: 0.8202


In [495]:
score = model_1.evaluate(x_test, y_test)
print('測試資料的 loss:', score[0])
print('測試資料正確率:', score[1])

68668/68668 [==============================] - 2s 30us/step
測試資料的 loss: 0.43292066231113907
測試資料正確率: 0.82018795218348


#### Model 2

使用相對較難、具有分岐之深度學習網絡

In [497]:
x = Input(shape=(6,))
# 不同階層的函數
f_1 = Dense(400, activation='relu')
f_2 = Dense(200, activation='relu')
f_3 = Dense(200, activation='relu')
f_4 = Dense(100, activation='relu')
f_5 = Dense(100, activation='relu')
f_6 = Dense(400, activation='relu')
f_7 = Dense(200, activation='relu')
f_8 = Dense(200, activation='relu')
f_9 = Dense(100, activation='relu')
f_10 = Dense(3, activation='sigmoid')
# hiden layer
h_11 = f_1(x)
h_21 = f_2(h_11)
h_22 = f_3(h_11)
u1=concatenate([h_21,h_22])
h_31 = f_4(u1)
h_12 = f_6(x)
h_23 = f_7(h_12)
h_24 = f_8(h_12)
u2=concatenate([h_23,h_24])
h_32 = f_9(u2)
u3=concatenate([h_31,h_32])
h_4 = f_5(u3)
y = f_10(h_4)
model_2 = Model(x, y)
#model.summary()
#model training
model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_2t=model_2.fit(x_train, y_train, batch_size=256, epochs=20,validation_data = (x_test, y_test))

Train on 160222 samples, validate on 68668 samples
Epoch 1/20
160222/160222 [==============================] - 12s 73us/step - loss: 0.4458 - acc: 0.8186 - val_loss: 0.4289 - val_acc: 0.8205
Epoch 2/20
160222/160222 [==============================] - 11s 69us/step - loss: 0.4293 - acc: 0.8209 - val_loss: 0.4296 - val_acc: 0.8206
Epoch 3/20
160222/160222 [==============================] - 11s 69us/step - loss: 0.4274 - acc: 0.8208 - val_loss: 0.4287 - val_acc: 0.8206
Epoch 4/20
160222/160222 [==============================] - 11s 70us/step - loss: 0.4256 - acc: 0.8210 - val_loss: 0.4267 - val_acc: 0.8205
Epoch 5/20
160222/160222 [==============================] - 12s 72us/step - loss: 0.4249 - acc: 0.8209 - val_loss: 0.4274 - val_acc: 0.8208
Epoch 6/20
160222/160222 [==============================] - 12s 76us/step - loss: 0.4233 - acc: 0.8210 - val_loss: 0.4209 - val_acc: 0.8205
Epoch 7/20
160222/160222 [==============================] - 13s 79us/step - loss: 0.4215 - acc: 0.8210 - val_

In [498]:
score = model_2.evaluate(x_test, y_test)
print('測試資料的 loss:', score[0])
print('測試資料正確率:', score[1])

68668/68668 [==============================] - 5s 72us/step
測試資料的 loss: 0.40190687386221713
測試資料正確率: 0.8261732757751211


In [491]:
#backup
t = '''
#學生卡
dfr1 = dfr[dfr['卡種']==6]
dfr1 = dfr1.drop(['卡種'],axis = 1)
dfr1y = np.array([1]*dfr1.shape[0])
dfr1t,dfr1test,dfr1yl,dfr1ytest= train_test_split(dfr1,dfr1y,test_size=0.3)   #切分資料為 train test

#普通卡
dfr2 = dfr[dfr['卡種']==1]
dfr2 = dfr2.drop(['卡種'],axis = 1) 
dfr2y = np.array([0]*dfr2.shape[0])
dfr2t,dfr2test,dfr2yl,dfr2ytest= train_test_split(dfr2,dfr2y,test_size=0.3)

#其他卡種
dfr3 = dfr[(dfr['卡種']!=1) & (dfr['卡種']!=6)]
dfr3 = dfr3.drop(['卡種'],axis = 1) 
dfr3y = np.array([0]*dfr3.shape[0])
dfr3t,dfr3test,dfr3yl,dfr3ytest= train_test_split(dfr3,dfr3y,test_size=0.3)

x_train = pd.concat([dfr1t,dfr2t,dfr3t,dfn1t,dfn2t,dfn3t,dfn4t],axis=0)    #部分合理卡 + 部分type1234
y_train = np.concatenate([dfr1yl,dfr2yl,dfr3yl,dfn1yl,dfn2yl,dfn3yl,dfn4yl])
x_test =  pd.concat([dfr1test,dfr2test,dfr3test],axis=0)   #其他部分合理卡
y_test = np.concatenate([dfr1ytest,dfr2ytest,dfr3ytest])
y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)'''